In [ ]:
import duckdb
from lewidi_lib import list_preds

pdf = (
    list_preds()
    .query("exists and run_name == '1000ex_10loops' and template_id == '60'")
    .astype({"preds_file": "str"})
)
con = duckdb.connect()
df = con.sql(
    f"""
    SELECT filename as preds_file, n_input_tokens, n_output_tokens 
    FROM read_parquet({[str(f) for f in pdf["preds_file"]]}, filename=true)
    WHERE success = true
    """
).df()
joint = pdf.merge(df, on="preds_file", how="left")
long = joint.melt(
    id_vars=["dataset", "model_id"],
    value_vars=["n_input_tokens", "n_output_tokens"],
    var_name="part",
    value_name="n_tokens",
)
long.head()

In [ ]:
# joint.groupby(["dataset", "model_id"], as_index=False)[["n_rows", "n_input_tokens", "n_output_tokens"]].sum()
import seaborn as sns

sns.catplot(
    long.query("part == 'n_output_tokens'"),
    y="n_tokens",
    x="dataset",
    hue="model_id",
    kind="bar",
)

In [ ]:
# why the hell does 8B compute so much on VariErrNLI?
fgrid = sns.FacetGrid(
    long.query("dataset == 'VariErrNLI' and part == 'n_output_tokens'"),
    row="model_id",
    height=1,
    aspect=5
)
fgrid.map(sns.histplot, "n_tokens")

In [ ]:
import pandas as pd
rdf = pd.read_parquet(pdf.query("dataset == 'VariErrNLI' and model_id == 'Qwen/Qwen3-8B'")["preds_file"].values[0])
print(rdf.sort_values("n_output_tokens", ascending=False)["reasoning"].values[0])